# Imports

In [16]:
import json
import os
import pandas as pd
import numpy as np

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.training import *
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

from sparknlp_jsl.training import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

import sparknlp_jsl
from pyspark.sql import SparkSession

from sparknlp.util import *

# Installation

In [ ]:
with open('../spark_nlp_for_healthcare-3.3.4.json', 'r') as f:
    license_keys = json.load(f)
    for k,v in license_keys.items(): 
        %set_env $k=$v

In [3]:
locals().update(license_keys)

# Session management

In [4]:
params = {"spark.driver.memory":"64G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"6000M"}

spark = sparknlp_jsl.start(secret=SECRET, params=params)

In [5]:
spark

In [6]:
sparknlp_jsl.version()

'3.3.4'

# Checking in an inference pipeline

In [50]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

word_embeddings = RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["document", "token"])\
    .setOutputCol("roberta_embeddings")

ner = MedicalNerModel.pretrained("roberta_ner_diag_proc","es","clinical/models")\
    .setInputCols("sentence","token","roberta_embeddings")\
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

snomed_prepro_pipeline = Pipeline(stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ner,
    ner_converter
    ])

empty = spark.createDataFrame([['']]).toDF("text")

p_model = snomed_prepro_pipeline.fit(empty)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]
roberta_ner_diag_proc download started this may take some time.
Approximate size to download 15.6 MB
[OK!]


In [54]:
test_sentence = 'Mujer de 28 años con antecedentes de diabetes mellitus gestacional diagnosticada ocho años antes de la presentación y posterior diabetes mellitus tipo dos (DM2), un episodio previo de pancreatitis inducida por HTG tres años antes de la presentación, asociado con una hepatitis aguda, y obesidad con un índice de masa corporal (IMC) de 33,5 kg / m2, que se presentó con antecedentes de una semana de poliuria, polidipsia, falta de apetito y vómitos. Dos semanas antes de la presentación, fue tratada con un ciclo de cinco días de amoxicilina por una infección del tracto respiratorio. Estaba tomando metformina, glipizida y dapagliflozina para la DM2 y atorvastatina y gemfibrozil para la HTG. Había estado tomando dapagliflozina durante seis meses en el momento de la presentación. El examen físico al momento de la presentación fue significativo para la mucosa oral seca; significativamente, su examen abdominal fue benigno sin dolor a la palpación, protección o rigidez. Los hallazgos de laboratorio pertinentes al ingreso fueron: glucosa sérica 111 mg / dl, bicarbonato 18 mmol / l, anión gap 20, creatinina 0,4 mg / dl, triglicéridos 508 mg / dl, colesterol total 122 mg / dl, hemoglobina glucosilada (HbA1c) 10%. y pH venoso 7,27. La lipasa sérica fue normal a 43 U / L. Los niveles séricos de acetona no pudieron evaluarse ya que las muestras de sangre se mantuvieron hemolizadas debido a una lipemia significativa. La paciente ingresó inicialmente por cetosis por inanición, ya que refirió una ingesta oral deficiente durante los tres días previos a la admisión. Sin embargo, la química sérica obtenida seis horas después de la presentación reveló que su glucosa era de 186 mg / dL, la brecha aniónica todavía estaba elevada a 21, el bicarbonato sérico era de 16 mmol / L, el nivel de triglicéridos alcanzó un máximo de 2050 mg / dL y la lipasa fue de 52 U / L. Se obtuvo el nivel de β-hidroxibutirato y se encontró que estaba elevado a 5,29 mmol / L; la muestra original se centrifugó y la capa de quilomicrones se eliminó antes del análisis debido a la interferencia de la turbidez causada por la lipemia nuevamente. El paciente fue tratado con un goteo de insulina para euDKA y HTG con una reducción de la brecha aniónica a 13 y triglicéridos a 1400 mg / dL, dentro de las 24 horas. Se pensó que su euDKA fue precipitada por su infección del tracto respiratorio en el contexto del uso del inhibidor de SGLT2. La paciente fue atendida por el servicio de endocrinología y fue dada de alta con 40 unidades de insulina glargina por la noche, 12 unidades de insulina lispro con las comidas y metformina 1000 mg dos veces al día. Se determinó que todos los inhibidores de SGLT2 deben suspenderse indefinidamente. Tuvo un seguimiento estrecho con endocrinología post alta.'
res = p_model.transform(spark.createDataFrame(pd.DataFrame({'text': [test_sentence]})))

In [67]:
c2doc = Chunk2Doc() \
    .setInputCols("ner_chunk") \
    .setOutputCol("sentence")

chunk_tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

chunk_word_embeddings = RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("ner_chunk_word_embeddings")

chunk_embeddings = SentenceEmbeddings() \
    .setInputCols(["sentence", "ner_chunk_word_embeddings"]) \
    .setOutputCol("ner_chunk_embeddings") \
    .setPoolingStrategy("AVERAGE")

er = SentenceEntityResolverModel.pretrained("robertaresolve_snomed", "es", "clinical/models")\
    .setInputCols(["sentence", "ner_chunk_embeddings"]) \
    .setOutputCol("snomed_code") \
    .setDistanceFunction("EUCLIDEAN")

snomed_resolve_pipeline = Pipeline(stages = [
    c2doc,
    chunk_tokenizer,
    chunk_word_embeddings,
    chunk_embeddings,
    er
    ])


roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]


In [68]:
p_infer_model = snomed_resolve_pipeline.fit(res.select('ner_chunk'))

In [69]:
final_res = p_infer_model.transform(res.select('ner_chunk'))

In [80]:
chunk = 'ner_chunk'
output_col='snomed_code'

pd.set_option('display.max_colwidth', 0)

df = final_res.select(F.explode(F.arrays_zip(chunk+'.begin', chunk+'.end', chunk+'.result', 
                                                   chunk+'.metadata', 
                                                   output_col+'.result', 
                                                   output_col+'.metadata')).alias("cols")) \
                             .select(F.expr("cols['0']").alias("begin"),
                                     F.expr("cols['1']").alias("end"),
                                     F.expr("cols['3']['sentence']").alias("sent_id"),
                                     F.expr("cols['2']").alias("ner_chunk"),
                                     F.expr("cols['3']['entity']").alias("entity"), 
                                     F.expr("cols['4']").alias("snomed_code"),
                                     F.expr("cols['5']['all_k_resolutions']").alias("snomed_texts"),                                     
                                     ).toPandas()

In [81]:
df

,begin,end,sent_id,ner_chunk,entity,snomed_code,snomed_texts
0,37,65,0,diabetes mellitus gestacional,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
1,128,153,0,diabetes mellitus tipo dos,DIAGNOSTICO,46635009,diabetes mellitus tipo I:::diabetes mellitus tipo II:::diabetes mellitus insulinodependiente tipo IA:::diabetes mellitus tipo II lábil:::diabetes mellitus tipo 1 lábil:::diabetes mellitus tipo MODY:::diabetes mellitus insulinodependiente tipo IB:::coma hipoglucémico en diabetes mellitus tipo I:::diabetes mellitus tipo 2 preexistente:::diabetes mellitus tipo 2 sin complicaciones:::cetoacidosis en diabetes mellitus tipo II:::diabetes mellitus tipo II complicada:::embarazo y diabetes mellitus tipo 2:::diabetes mellitus tipo 2 con gangrena:::trastorno debido a diabetes mellitus tipo II
2,184,195,0,pancreatitis,DIAGNOSTICO,75694006,pancreatitis:::pancreatalgia:::pancreatoscopio:::pancreatina:::pancreatina:::pancreatenfraxis:::pancreatitis recurrente:::pancreatitis recidivante:::pancreatitis recidivante:::pancreatitis necrosante:::pancreatitis apopléjica:::pancreatitis indolora:::pancreaticocistoduodenostomía:::pancreatitis hereditaria:::pancreaticoduodenostomía:::pancreaticogastrostomía:::pancreatitis metabólica:::pancreaticocistoyeyunostomía:::pancreatectomía:::pancreatitis aguda:::pancreatitis aguda
3,267,282,0,"hepatitis aguda,",DIAGNOSTICO,37871000,hepatitis aguda:::hepatitis C aguda (trastorno):::conjuntivitis aguda:::hepatitis vírica aguda:::hepatitis E aguda (trastorno):::hepatitis vírica aguda tipo B:::hepatitis B aguda con hepatitis D:::hepatitis focal aguda (trastorno):::linfangitis aguda del pie:::hepatitis tóxica aguda:::fiebre Q aguda (trastorno):::artritis aguda (trastorno):::leucemia aguda (categoría):::hepatitis viral aguda tipo A (trastorno):::insuficiencia aguda (hallazgo)
4,286,293,0,obesidad,DIAGNOSTICO,5476005,obesidad:::obesidad:::carcinoma:::carcinoma:::infarto:::activo:::bilis:::bilis:::celulosa:::celulosa:::ARN:::dieta:::hombre:::hombre:::adrenalina:::adrenalina:::adrenalina:::adrenalina:::adrenalina:::adrenalina+inotrópicos:::distrofia
5,302,324,0,índice de masa corporal,DIAGNOSTICO,162859006,"índice de masa corporal:::índice de masa corporal:::índice de masa corporal normal:::índice de masa corporal aumentado:::índice de masa corporal disminuido:::índice de masa corporal - hallazgo:::centilo del índice de masa corporal:::índice de masa corporal, alto K/M2:::índice de masa corporal 25-29, sobrepeso:::cálculo del índice de masa corporal:::índice de masa corporal, bajo K/M2:::índice de masa corporal, normal K/M2:::índice de masa corporal 30+, obesidad:::índice de masa corporal de 30,00 a 34,99:::índice de masa (propiedad):::analizador de índice de masa corporal bioeléctrico:::índice de masa corporal de 35,00 a 39,99:::índice de masa corporal, 40+ - obesidad grave:::índice de masa corporal inferior a 20"
6,327,329,0,IMC,DIAGNOSTICO,60621009,IMC:::mL:::mg/comida:::mg/kg/día:::mg/m2:::mg/alimento:::mg/día/kg:::mg/kg/h:::mg/dl:::mg/día:::mg/m2/día:::mg/m2/h:::mg/ml:::mg/intercambio:::mg/l:::mg/disparo:::masa:::masa:::masa:::masa
7,399,406,0,poliuria,DIAGNOSTICO,56574000,poliuria:::poliuria:::polifagia:::polifagia:::disuria:::frecuencia miccional y poliuria:::proteinuria:::rectorragia:::rectorragia:::proteinuria prerrenal:::proteinuria colicuativa:::piuria:::hematuria macroscópica:::h

# Getting examples

In [8]:
!unzip codiesp.zip

Archive:  codiesp.zip
   creating: final_dataset_v4_to_publish/
  inflating: final_dataset_v4_to_publish/README.txt  
   creating: final_dataset_v4_to_publish/dev/
  inflating: final_dataset_v4_to_publish/dev/devD.tsv  
  inflating: final_dataset_v4_to_publish/dev/devP.tsv  
  inflating: final_dataset_v4_to_publish/dev/devX.tsv  
   creating: final_dataset_v4_to_publish/dev/text_files_en/
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S0004-06142005000900016-1.txt  
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S0004-06142005001000011-1.txt  
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S0004-06142006000200011-1.txt  
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S0004-06142006000500002-3.txt  
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S0004-06142006000500002-4.txt  
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S0004-06142006000600015-1.txt  
  inflating: final_dataset_v4_to_publish/dev/text_files_en/S00

In [87]:
rdr = CodiEspReader()
###TEST DATA
test_data = rdr.readDatasetTaskX(spark,
                            "./final_dataset_v4_to_publish/test/testX.tsv",
                           "./final_dataset_v4_to_publish/test/text_files/")

In [88]:
test_data.select('text').show()

+--------------------+
|                text|
+--------------------+
|Mujer de 46 años ...|
|Mujer de raza mes...|
|Se presenta un ca...|
|Varón de 22 años ...|
|Niño de 13 años r...|
|Se trata de un va...|
|Niño de nueve año...|
|Se trata de un pa...|
|Varón de 75 años ...|
|Paciente de 36 añ...|
|Presentamos el ca...|
|Varón de 70 años,...|
|Presentamos el ca...|
|Paciente varón de...|
|Varón de 56 años ...|
|Varón de 38 años ...|
|Mujer de 32 años,...|
|Varón de 30 años ...|
|Mujer de 56 años ...|
|En el año 2000, a...|
+--------------------+
only showing top 20 rows



In [89]:
all_sentences = test_data.collect()

In [90]:
sentences = []
for i in range(0,100):
    sentences.append(all_sentences[i].text)

In [91]:
sentences[0]

'Mujer de 46 años de edad, con antecedente de endometriosis ovárica, a quien se practicó una histerectomía abdominal con doble anexectomía por endometriosis 45 días antes de acudir a consulta, remitida por su ginecólogo por presentar desde el postoperatorio inmediato dolor continuo en región iliaca y sacra derecha que mejoraba con antiinflamatorios y por objetivarse en una ecografía abdominal una discreta ectasia renal. A la exploración la paciente presenta dolor y defensa a la palpación profunda de fosa lumbar derecha y dolorimiento a la percusión y palpación de fosa lumbar izquierda, sugestivo de patología retroperitoneal.\nSe realiza urografía intravenosa (UIV) y Tomografía computerizada (TAC) simultáneos, observándose riñón derecho funcionante con retraso en la captación y eliminación de contraste y borramiento de la línea de psoas derecho. En la TAC se objetiva imagen lobulada de contenido líquido y con aparente extravasación de contraste, estando el uréter incluido en dicha forma

In [106]:
sel_sentences = [5, 6, 53, 60, 80]

In [107]:
INPUT_FILE_PATH = "./demo/inputs"
OUTPUT_FILE_PATH = "./demo/outputs"

In [102]:
!mkdir -p $INPUT_FILE_PATH
!mkdir -p $OUTPUT_FILE_PATH

In [108]:
counter = 1
for i in sel_sentences:
    test_sentence = sentences[i]
    open(os.path.join(INPUT_FILE_PATH,'Example'+str(counter)+'.txt'), 'w').write(test_sentence[:min(len(test_sentence)-10, 100)]+'... \n'+test_sentence)
    r = p_model.transform(spark.createDataFrame(pd.DataFrame({'text': [test_sentence]})))
    n = p_infer_model.transform(r.select('ner_chunk'))
    
    chunk = 'ner_chunk'
    output_col='snomed_code'

    df = n.select(F.explode(F.arrays_zip(chunk+'.begin', chunk+'.end', chunk+'.result', 
                                                           chunk+'.metadata', 
                                                           output_col+'.result', 
                                                           output_col+'.metadata')).alias("cols")) \
                                     .select(F.expr("cols['0']").alias("begin"),
                                             F.expr("cols['1']").alias("end"),
                                             F.expr("cols['3']['sentence']").alias("sent_id"),
                                             F.expr("cols['2']").alias("chunk"),
                                             F.expr("cols['3']['entity']").alias("entity_type"), 
                                             F.expr("cols['4']").alias("snomed_code"),
                                             F.expr("cols['5']['all_k_resolutions']").alias("snomed_texts"),                                     
                                             ).toPandas()

    df['snomed_description'] = df['snomed_texts'].apply(lambda x: x.split(':')[0])
    df[['chunk','begin','end','entity_type','snomed_code','snomed_description']].to_csv(os.path.join(OUTPUT_FILE_PATH,'Example'+str(counter)+'.csv'), index=False)
    counter +=1
